## Learning Black-Scholes Equation for Vanilla Options

In [257]:
import numpy as np
import torch
import csv

In [258]:
dataset = []
with open('../data/black_scholes.csv', newline='\n') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  for row in reader:
    dataset.append(row)

train_set, test_set = np.array(dataset[:50000], dtype=float), np.array(dataset[50000:60000], dtype=float)
train_features, train_labels = torch.from_numpy(train_set[:, 0:5]), torch.from_numpy(train_set[:, 5]).reshape(-1, 1)
test_features, test_labels = torch.from_numpy(test_set[:, 0:5]), torch.from_numpy(test_set[:, 5]).reshape(-1, 1)

In [259]:
batch_size = 64
train_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_features, train_labels),
                                         batch_size,
                                         shuffle=True)
test_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_features, test_labels),
                                        batch_size,
                                        shuffle=False)


In [260]:
num_inputs = 5
num_hidden = 100
num_outputs = 1
dropout = .2
model = torch.nn.Sequential(torch.nn.Linear(num_inputs, num_hidden),
                            torch.nn.LeakyReLU(),
                            torch.nn.Dropout(dropout),
                            torch.nn.Linear(num_hidden, num_hidden),
                            torch.nn.ELU(),
                            torch.nn.Dropout(dropout),
                            torch.nn.Linear(num_hidden, num_hidden),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(dropout),
                            torch.nn.Linear(num_hidden, num_hidden),
                            torch.nn.ELU(),
                            torch.nn.Dropout(dropout),
                            torch.nn.Linear(num_hidden, num_outputs))

In [261]:
loss = torch.nn.MSELoss()

In [262]:
learning_rate = 0.002
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [263]:
num_epochs = 10
for epoch in range(num_epochs):
  for X, y in train_iter:
    l = loss(model(X.float()), y.float())

    optimizer.zero_grad()
    l.backward()
    optimizer.step()

  l = loss(model(train_features.float()), train_labels.float())
  print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.741730
epoch 2, loss 0.510371
epoch 3, loss 0.422268
epoch 4, loss 0.548006
epoch 5, loss 1.449436
epoch 6, loss 0.343896
epoch 7, loss 0.376971
epoch 8, loss 0.346611
epoch 9, loss 0.321213
epoch 10, loss 0.292195


In [264]:
with torch.no_grad():
  print(model(train_features[:5].float()))
  print(train_labels[:5].float())

tensor([[ 7.9111e-03],
        [-3.7489e-03],
        [ 2.3202e-01],
        [ 3.1241e+01],
        [-1.9131e-02]])
tensor([[0.0000e+00],
        [1.0551e-02],
        [2.4765e-01],
        [3.4387e+01],
        [5.1302e-14]])


In [267]:
with torch.no_grad():
  l = loss(model(test_features.float()), test_labels.float())
  print(f'loss {l:f}')

loss 0.307934


In [ ]:
torch.save(model.state_dict(), '../save/black_scholes.ckpt')